# Dynamic Scraping Example

### Import the libraries

In [ ]:
#!pip install -U selenium
#!pip install -U webdriver-manager

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver = webdriver.Chrome()

In [ ]:
# test the driver
driver.get('http://www.nu.nl')

In [1]:
# import relevant libraries
from bs4 import BeautifulSoup
import json
import os
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException    

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# this to get rid of the pesky "Chrome is being controlled by automated test software" that messed up the cool autoscroll

options = webdriver.ChromeOptions()
options.add_argument('disable-infobars')
#options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# get latest Chrome driver
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver = webdriver.Chrome()

### Select a station

In [ ]:
def select_station():
  # click on select station
  driver.find_element('id', 'pbs_loc_name').click()

  time.sleep(1)
  select = Select(driver.find_element('id', 'regionSelect'))
  select.select_by_value('NY')

  time.sleep(1)
  driver.find_element('id','serchByRegion').click()

  time.sleep(1)
  driver.find_element('id','WNET').click()

  time.sleep(1)
  driver.find_element('id','confirmStation').click()

# the url we are going to scrape
url = "https://www.pbs.org/"
driver.get(url)

# wait
time.sleep(5)

# choose a station and 
select_station()

# wait
time.sleep(2)

### Select a category and scroll down

In [ ]:
# the category we are interested in
genre = 'culture'

# get the url
url = 'https://www.pbs.org/shows/?search=&genre=' + genre
driver.get(url)

In [ ]:
# so I figured out why this did not work during the presentation. 
# Selenium adds a nice "Chrome is being controlled by automated test software" which never bothered this part of the code before
# And I probably closed that bar while testing the code. It works now!

# Get scroll height
last_height = driver.execute_script('return document.body.scrollHeight')

while True:
  # Scroll down to bottom
  driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

  # Wait to load page
  time.sleep(6)

  # Calculate new scroll height and compare with last scroll height
  new_height = driver.execute_script('return document.body.scrollHeight')
  if new_height == last_height:
    break
  last_height = new_height

### Get list of pages and download each show

In [ ]:
# get the page source from Selenium
page_source = driver.page_source

# parse through BS
soup = BeautifulSoup(page_source, 'html.parser')

shows = []

# get all tiles
items = soup.find_all('article')

# iterate through tiles
for item in items:
  link = item.find('a')['href']
  name = link.replace('/show/', '').replace('/', '')
  dir = 'data/'+genre

  show_url = 'http://www.pbs.org' + link
  file = dir + '/' + name + '.html'
  if os.path.isfile(file) == False:
    print('Retrieving', show_url)    
    driver.get(show_url)
    
    # PBS tends to block at one point. Sleep 10 is good
    time.sleep(10)
    
    # save
    with open(file, 'w') as f:
      f.write(driver.page_source)